In [2]:
import os
import itertools
from ast import literal_eval

import numpy as np
import pandas as pd

DATA_FOLDER = os.path.abspath('data')
DATASET_FILE = os.path.join(DATA_FOLDER, 'TA_restaurants_curated.csv')

PROCESSED_DATA_FOLDER = os.path.join(DATA_FOLDER, 'processed')

In [3]:
# load dataframe from csv file
def load_df(file):

    if not os.path.exists(file): 
        print('File doesn\'t exist:', file)
        return

    df = pd.read_csv(file)
    return df


def transform_df(df):

    # drop redundant columns
    df.drop(
        columns=['Reviews', 'URL_TA', 'ID_TA'], 
        inplace=True
    )

    # drop useless rows
    df.dropna(
        subset=['Cuisine Style', 'Ranking', 'Rating', 'Price Range', 'Number of Reviews'], 
        how='all', inplace=True
    )

    def literal_return(x):
        try: return literal_eval(str(x)) 
        except Exception as e: return []

    # convert string-enclosed lists to actual lists
    df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: literal_return(x))

    return df


# save dataframe to csv file
def save_df(df, folder, file_name):

    if os.path.isdir(folder):
        file_path = os.path.join(folder, file_name)
        df.to_csv(file_path, mode='w', header=True, sep=',', lineterminator='', index=True)


df = load_df(DATASET_FILE)
df = transform_df(df)

print('total rows:', len(df))
df.head()

#save_df(df, DATA_FOLDER, 'TA_restaurants_curated_cleaned.csv')

total rows: 121055


,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews
0,0,Martine of Martine's Table,Amsterdam,"[French, Dutch, European]",1.0,5.0,$$ - $$$,136.0
1,1,De Silveren Spiegel,Amsterdam,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,$$$$,812.0
2,2,La Rive,Amsterdam,"[Mediterranean, French, International, Europea...",3.0,4.5,$$$$,567.0
3,3,Vinkeles,Amsterdam,"[French, European, International, Contemporary...",4.0,5.0,$$$$,564.0
4,4,Librije's Zusje Amsterdam,Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,$$$$,316.0


In [4]:
# distinct Cities
distinct_cities = df['City'].unique()
print('Distinct Cities Total:', len(distinct_cities))
print('Distinct Cities:', distinct_cities)

# distinct Cuisine Styles
distinct_styles = sorted(set(itertools.chain.from_iterable(df['Cuisine Style'])))
print('Distinct Cuisine Styles Total:', len(distinct_styles))
print('Distinct Cuisine Styles:', distinct_styles)

# restaurant types
styles_rest_types = [
    'Bar', 'Brew Pub', 'Cafe', 'Diner', 'Gastropub', 
    'Pub', 'Steakhouse', 'Wine Bar'
]

# food types
styles_food_types = [
    'Balti', 'Barbecue', 'Contemporary', 'Delicatessen', 'Fast Food', 
    'Fusion', 'Gluten Free Options', 'Grill', 'Halal', 'Healthy', 'Kosher', 
    'Pizza', 'Seafood', 'Soups', 'Street Food', 'Sushi', 
    'Vegan Options', 'Vegetarian Friendly'
]

# regions
styles_regions = [
    'African', 'Arabic', 'Asian', 'Cajun & Creole', 'Caribbean', 'Caucasian', 
    'Central American', 'Central Asian', 'Central European', 'Eastern European', 
    'European', 'International', 'Latin', 'Mediterranean', 'Middle Eastern', 
    'Minority Chinese', 'Native American', 'Polynesian', 'South American', 
    'Southwestern', 'Xinjiang', 'Yunnan'
]

# countries
styles_countries  = sorted(set(distinct_styles.copy()) - {*styles_rest_types, *styles_food_types, *styles_regions})
#print(styles_countries)

cuisine_styles = {
    'rest_type': styles_rest_types, 
    'food_type': styles_food_types, 
    'region': styles_regions, 
    'country': styles_countries, 
}

Distinct Cities Total: 31
Distinct Cities: ['Amsterdam' 'Athens' 'Barcelona' 'Berlin' 'Bratislava' 'Brussels'
 'Budapest' 'Copenhagen' 'Dublin' 'Edinburgh' 'Geneva' 'Hamburg'
 'Helsinki' 'Krakow' 'Lisbon' 'Ljubljana' 'London' 'Luxembourg' 'Lyon'
 'Madrid' 'Milan' 'Munich' 'Oporto' 'Oslo' 'Paris' 'Prague' 'Rome'
 'Stockholm' 'Vienna' 'Warsaw' 'Zurich']
Distinct Cuisine Styles Total: 127
Distinct Cuisine Styles: ['Afghani', 'African', 'Albanian', 'American', 'Arabic', 'Argentinean', 'Armenian', 'Asian', 'Australian', 'Austrian', 'Azerbaijani', 'Balti', 'Bangladeshi', 'Bar', 'Barbecue', 'Belgian', 'Brazilian', 'Brew Pub', 'British', 'Burmese', 'Cafe', 'Cajun & Creole', 'Cambodian', 'Canadian', 'Caribbean', 'Caucasian', 'Central American', 'Central Asian', 'Central European', 'Chilean', 'Chinese', 'Colombian', 'Contemporary', 'Croatian', 'Cuban', 'Czech', 'Danish', 'Delicatessen', 'Diner', 'Dutch', 'Eastern European', 'Ecuadorean', 'Egyptian', 'Ethiopian', 'European', 'Fast Food', 'Filipin

In [5]:
"""
### cuisine_styles.csv
    style_name, is_rest_type, is_food_type, is_region, is_country
"""

def create_cuisince_stlyes_df():

    cuisine_styles_df = pd.DataFrame(
        columns=[
            'style_name', 
            'is_rest_type', 'is_food_type', 'is_region', 'is_country'
        ]
    )

    cuisine_styles_df['style_name'] = distinct_styles
    cuisine_styles_df[['is_rest_type', 'is_food_type', 'is_region', 'is_country']] = 0

    # update binarized is_x_type columns
    for style_type in cuisine_styles:
        for style_name in cuisine_styles[style_type]:
            cuisine_styles_df['is_'+style_type] = np.where(
                cuisine_styles_df['style_name'] == style_name, 
                1, cuisine_styles_df['is_'+style_type]
            )

    return cuisine_styles_df


cuisine_styles_df = create_cuisince_stlyes_df()
cuisine_styles_df.head()

#save_df(cuisine_styles_df, PROCESSED_DATA_FOLDER, 'cuisine_styles.csv')

,style_name,is_rest_type,is_food_type,is_region,is_country
0,Afghani,0,0,0,1
1,African,0,0,1,0
2,Albanian,0,0,0,1
3,American,0,0,0,1
4,Arabic,0,0,1,0


In [6]:
def total_cuisine_style_restaurants_per_city(df, city_name):

    styles = list(itertools.chain.from_iterable(df['Cuisine Style'].loc[df.City==city_name]))
    distinct_styles = set(styles)
    #print('distinct styles:', len(distinct_styles))

    #for style_name in distinct_styles:
    #    print(f'{style_name}:', styles.count(style_name))

    return len(distinct_styles)


#total_cuisine_style_restaurants_per_city(df, 'Amsterdam')


"""
### cities.csv
    city_name, total_restaurants, 
    total_cuisine_styles, total_rated, total_reviewed
    total_reviews, average_rating, average_reviews
"""

def create_cities_df(df):

    cities_df = pd.DataFrame(
        columns=[
            'city_name', 'total_restaurants', 
            'total_cuisine_styles', 'total_rated', 'total_reviewed', 
            'total_reviews', 'average_rating', 'average_reviews', 
        ]
    )

    # summarized data for each city
    for city_name in distinct_cities:

        temp_df = df.loc[df['City'] == city_name]

        total_restaurants = len(temp_df)
        total_cuisine_styles = int(total_cuisine_style_restaurants_per_city(df, city_name))
        total_rated = temp_df.loc[temp_df['Rating'] > 0]
        average_rating = round(total_rated['Rating'].mean(), 2)
        total_reviewed = temp_df.loc[temp_df['Number of Reviews'] > 0]
        total_reviews = int(total_reviewed['Number of Reviews'].sum())
        average_reviews = int(total_reviewed['Number of Reviews'].mean())

        cities_df.loc[-1] = [
            city_name, total_restaurants, total_cuisine_styles, 
            len(total_rated), len(total_reviewed), 
            total_reviews, average_rating, average_reviews
        ]
        cities_df.index = cities_df.index + 1
        cities_df = cities_df.sort_index()

    cities_df.sort_values(by=['city_name'], inplace=True)
    cities_df.reset_index(drop=True, inplace=True)

    # first row, contains aggregated data from all cities combined
    cities_df.loc[-1] = [
        'ALL', 
        cities_df['total_restaurants'].sum(), 
        len(distinct_styles), 
        cities_df['total_rated'].sum(), 
        cities_df['total_reviewed'].sum(), 
        cities_df['total_reviews'].sum(), 
        round(cities_df['average_rating'].mean(), 2), 
        int(cities_df['average_reviews'].mean())
    ]
    cities_df.index = cities_df.index + 1
    cities_df = cities_df.sort_index()

    return cities_df


cities_df = create_cities_df(df)
cities_df.head()

#save_df(cities_df, PROCESSED_DATA_FOLDER, 'cities.csv')

,city_name,total_restaurants,total_cuisine_styles,total_rated,total_reviewed,total_reviews,average_rating,average_reviews
0,ALL,121055,127,115856,108183,13542887,4.01,112
1,Amsterdam,3386,97,3243,3061,414444,4.13,135
2,Athens,1887,78,1787,1643,166935,4.21,101
3,Barcelona,8165,105,7790,7264,1020548,3.97,140
4,Berlin,6694,109,6371,5800,406937,4.13,70


In [233]:
"""
### city_cuisine_styles.csv
    city_name, style_name, total_restaurants, 
    total_rated, total_reviewed, rating_sum, 
    average_rating, total_reviews, average_reviews
"""

def create_city_cuisine_styles_df(df, cities: list):

    city_cuisine_styles_df = pd.DataFrame(
        columns=[
            'city_name', 'style_name', 'total_restaurants', 
            'total_rated', 'total_reviewed', 'rating_sum', 
            'average_rating', 'total_reviews', 'average_reviews', 
        ]
    )

    for city_name in cities:

        # add all distinct cuisine style rows for each city
        for style_name in distinct_styles:

            city_cuisine_styles_df.loc[-1] = [
                city_name, style_name, 0, 
                0, 0, 0, 
                0, 0, 0, 
            ]
            city_cuisine_styles_df.index = city_cuisine_styles_df.index + 1
            city_cuisine_styles_df.sort_index()

        city_cuisine_styles_df = city_cuisine_styles_df.sort_values(by=['city_name'])
        city_cuisine_styles_df = city_cuisine_styles_df.reset_index(drop=True)

        # loop over city rows, collect and store data in new df
        for index, row in df.loc[df['City']==city_name].iterrows():

            # get the data for each cuisine style
            for style_name in row['Cuisine Style']:

                row_index = city_cuisine_styles_df.index[
                    (city_cuisine_styles_df['city_name'] == row['City']) &
                    (city_cuisine_styles_df['style_name'] == style_name) 
                ]

                if len(row_index)==1:
                    row_index = row_index[0]

                    city_cuisine_styles_df.loc[row_index, 'total_restaurants'] += 1

                    if row['Rating'] > 0:
                        city_cuisine_styles_df.loc[row_index, 'total_rated'] += 1
                        city_cuisine_styles_df.loc[row_index, 'rating_sum'] += row['Rating'] 

                    if row['Number of Reviews'] > 0:
                        city_cuisine_styles_df.loc[row_index, 'total_reviewed'] += 1
                        city_cuisine_styles_df.loc[row_index, 'total_reviews'] += row['Number of Reviews']

        # drop useless rows
        # (= rows without collected data for a city's cuisine style)
        city_cuisine_styles_df.drop(
            city_cuisine_styles_df[city_cuisine_styles_df['total_restaurants'] == 0].index, 
            inplace=True
        )

        # populate 'average' data columns
        city_cuisine_styles_df['average_rating'] = city_cuisine_styles_df.apply(
            lambda x: round(x.rating_sum / x.total_rated, 2) 
                if x.rating_sum > 0 and x.total_rated > 0 else 0, 
            axis=1
        )
        city_cuisine_styles_df['average_reviews'] = city_cuisine_styles_df.apply(
            lambda x: int(x.total_reviews / x.total_reviewed) 
                if x.total_reviews > 0 and x.total_reviewed > 0 else 0, 
            axis=1
        )

    # top rows containing aggregated data for all cities' cuisine styles combined
    for style_name in distinct_styles:
        
        style_df = city_cuisine_styles_df.loc[city_cuisine_styles_df.style_name==style_name]

        city_cuisine_styles_df.loc[-1] = [
            'ALL', 
            style_name, 
            style_df['total_restaurants'].sum(), 
            style_df['total_rated'].sum(), 
            style_df['total_reviewed'].sum(), 
            style_df['rating_sum'].sum(), 
            round(style_df['rating_sum'].sum()/style_df['total_rated'].sum(), 2) 
                if style_df['total_rated'].sum() > 0 else 0, 
            style_df['total_reviews'].sum(), 
            int(style_df['total_reviews'].sum()/style_df['total_reviewed'].sum()) 
                if style_df['total_reviewed'].sum() > 0 else 0, 
        ]
        city_cuisine_styles_df.index = city_cuisine_styles_df.index + 1
        city_cuisine_styles_df = city_cuisine_styles_df.sort_index()

    city_cuisine_styles_df.drop(columns=['rating_sum'], inplace=True)

    return city_cuisine_styles_df


city_cuisine_styles_df = create_city_cuisine_styles_df(df, ['Amsterdam']) # SINGLE CITY: FOR TESTING PURPOSES
#city_cuisine_styles_df = create_city_cuisine_styles_df(df, distinct_cities)
city_cuisine_styles_df

#save_df(city_cuisine_styles_df, PROCESSED_DATA_FOLDER, 'city_cuisine_styles.csv')